# Implementation of agent-based simulations

In [4]:
@info("Loading packages")
using Pkg; Pkg.activate("../test")
using Plots, StatsPlots, Plots.Measures
using StatsBase
using Glob
using DataFrames, DataFramesMeta
using ProgressMeter
default(leg = false, lw = 1.5)

using ComponentArrays, StaticArrays
using Test
using Parameters
using Distributions
using OrdinaryDiffEq


# establishing type hierarchy
abstract type AbstractParams end
abstract type AbstractParamCollection end # an AbstractParamCollection contain a defined set of multiple AbstractParams instances
abstract type AbstractSpeciesParams <: AbstractParams end
abstract type AbstractGlobalParams <: AbstractParams end
abstract type AbstractGlobalODEParams <: AbstractParams end
abstract type AbstractABM end
abstract type AbstractAgent end

┌ Info: Loading packages
└ @ Main c:\Users\simon\Documents\Julia\Hydra\test\abmdev.ipynb:1
  Activating project at `c:\Users\simon\Documents\Julia\Hydra\test`


In [7]:
include("../src/DoseResponse.jl")
include("../src/Params.jl")
include("../src/Solvers.jl")
include("../src/StateVars.jl")
include("../src/IO.jl")
include("../src/ModelFunctions.jl")
include("../src/HydraODE.jl")
include("../src/ImpliedTraits.jl")
include("../src/Macros.jl")

@replicates

In [8]:
using Test

In [9]:
abstract type AbstractABM end

"""
Currently assumes that all agents are represented by a single agent type `AgentType`.
"""
mutable struct ABM <: AbstractABM
    p::AbstractParamCollection # parameter collection
    t::Float64 # current simulation time
    dt::Float64 # timestep
    euler::Function # definition of the euler function for the given dt
    saveat::Float64
    
    u::ComponentVector # global state variables
    du::ComponentVector # global derivatives

    AgentID_count::Int64 # cumulative number of agents in the simulation
    agents::AbstractVector # population of agents
    aout::AbstractVector # agent output
    mout::AbstractVector # model output

    """
    Instantiate ABM from param collection `p`. 
    """
    function ABM(
        p::A; 
        dt = 1/24, 
        saveat = 1,
        execute = true
        ) where A <: AbstractParamCollection

        abm = new() # initialize ABM object

        abm.p = p # store the parameters
        abm.t = 0. # initialize simulation time
        abm.dt = dt # timestep is given as keyword argument
        abm.euler = defeuler(dt) # Euler function for the given dt
        abm.saveat = saveat
        
        abm.u = init_substates_global(p) # initialize the global substates
        abm.du = similar(abm.u) # initialize the global derivatives
        abm.du.X_p = 0.
        abm.du.C_W .= 0.
        
        abm.AgentID_count = 0 # set agent count
        initialize_agents!(abm) # initailize the population of agents
        abm.aout = [] # initialize agent output
        abm.mout = [] # initialize model output

        if execute
            run!(abm) # run the model

            return prepare_output(abm) # prepare output in dataframes and return
        else
            return abm
        end
    end
end

ABM